<a href="https://colab.research.google.com/github/fyuuki0jp/colab_llm_training/blob/main/test_generation_gemma2_2b_jpn_it_reasoning_sft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uq transformers datasets bitsandbytes torch xformers torchinfo torchvision
!pip install --upgrade --force-reinstall torchvision
# !pip install -Uq unsloth
# !pip install -Uq unsloth-zoo
# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

  Using cached torchvision-0.21.0-cp311-cp311-manylinux1_x86_64.whl.metadata (6.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.wh

object address  : 0x78bfa4abffa0
object refcount : 2
object type     : 0x9d5ea0
object type name: KeyboardInterrupt
object repr     : KeyboardInterrupt()
lost sys.stderr
^C


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM,TextStreamer

model_name = "fyuuki0jp/gemma-2-2b-jpn-it_finetuning_sft"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,device_map='auto',use_cache=False)

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [2]:
prompt = """<start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください<end_of_turn>
<start_of_turn>user
A～Dの中で3人は正直者で、1人が嘘つきです。嘘つきはだれでしょう？

A：Dはうそをついている。

B：僕はうそをついていない。

C：Aはうそをついていない。

D：Bはうそをついている。<end_of_turn>
<start_of_turn>model
"""
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer)
model.max_seq_length = 8196
outputs = model.generate(**input_ids,max_new_tokens=2048,streamer=streamer, top_k=2,top_p=0.95,do_sample=True, temperature=0.6,repetition_penalty=1.1)

<bos><start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください<end_of_turn>
<start_of_turn>user
A～Dの中で3人は正直者で、1人が嘘つきです。嘘つきはだれでしょう？

A：Dはうそをついている。

B：僕はうそをついていない。

C：Aはうそをついていない。

D：Bはうそをついている。<end_of_turn>
<start_of_turn>model


/usr/local/lib/python3.11/dist-packages/torch/_inductor/compile_fx.py:194: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0306 23:47:45.112000 3118 torch/_inductor/utils.py:1137] [0/0] Not enough SMs to use max_autotune_gemm mode


<thinking>
## 3人の正直者がいる状況から、嘘つきを特定する手順

**ステップ1: A, B, C の発言を分析する**

* **A**: Dが嘘をつくことを示唆している。
* 

CUDAGraph supports dynamic shapes by recording a new graph for each distinct input size. Recording too many CUDAGraphs may lead to extra overhead. We have observed 51 distinct sizes. Please consider the following options for better performance: a) padding inputs to a few fixed number of shapes; or b) set torch._inductor.config.triton.cudagraph_skip_dynamic_graphs=True. Set torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit=None to silence this warning.


**B**: 「僕」は嘘をつかないと主張している。しかし、「僕が嘘をついている」という情報が与えられているため、この発言だけでは判断できない。
* **C**: Aが嘘をつかないことを示唆している。ただし、これは「嘘つき」である可能性がある。

**ステップ2: B と D が語る内容を考慮する**

* **B**: 「僕の嘘はつかない」という発言は、自分が嘘をつきないことを保証している。
* **D**: 「僕の嘘はついている」という発言は、自分自身を嘘つきとして位置づけている。

**ステップ3: これらの情報を組み合わせて考える**

* **B** は嘘をつかないと主張しており、その一方で**D** は自分の嘘を認めている。
* したがって、**D** は嘘つきであり、他の2人（A, C）は正直者であることが推測できる。

**最終的な結論:**  嘘つきはDである。



</thinking><output>D</output><end_of_turn><eos>


In [3]:
prompt = """<start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください<end_of_turn>
<start_of_turn>user
空が青い理由を教えて<end_of_turn>
<start_of_turn>model
"""
input_ids = tokenizer(prompt, return_tensors="pt").to('cuda')
streamer = TextStreamer(tokenizer)
outputs = model.generate(**input_ids,max_new_tokens=2048,streamer=streamer, top_k=2,top_p=0.95,do_sample=True, temperature=0.6,repetition_penalty=1.1)

<bos><start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください<end_of_turn>
<start_of_turn>user
空が青い理由を教えて<end_of_turn>
<start_of_turn>model
<thinking>
## 空が青くなる理由をステップバイステップで説明します。

1. **光と色**: 光は様々な波長を持つ電磁波から成り、人間の目には見える範囲内では赤、橙、黄、緑、青、藍、紫の7種類の色があります。
2. **光の散乱**: 太陽からの光は地球の大気を通過しますが、大気の分子や水蒸気などの粒子によって散乱されます。
3. **ブルーカラーの強い散乱**: ブルー色の光は他の色よりも強く散乱されるため、私たちが目にする色は青くなります。
4. **太陽の位置と時間**: 午前中や午後、特に夕暮れ時に太陽が低い位置にあるとき、より多くのブルー色の光が散乱され、空が青く見えます。



</thinking><output>空が青い理由は、太陽からの光が地球の大気を通過するときに、大気の分子や水蒸気などにより散乱されてしまうからです。</output><end_of_turn><eos>


In [4]:
prompt = """<start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください<end_of_turn>
<start_of_turn>user
地球温暖化の主な原因は何ですか？<end_of_turn>
<start_of_turn>model
"""
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer)
outputs = model.generate(**input_ids,max_new_tokens=2048,streamer=streamer, top_k=2,top_p=0.95,do_sample=True, temperature=0.6,repetition_penalty=1.1)

<bos><start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください<end_of_turn>
<start_of_turn>user
地球温暖化の主な原因は何ですか？<end_of_turn>
<start_of_turn>model
<thinking>
## 地球温暖化の原因を特定する手順

1. **問題文を理解する:** 問題は、地球温暖化の主な原因について尋ねています。

2. **知識を想起する:** 地球温暖化と環境問題に関する一般的な知識を思い浮かべます。

3. **主要な要因をリストアップする:** 
    * 気温の上昇
    * 大気中の二酸化炭素（CO2）の増加
    * 温室効果ガス（GHGs）の増加

4. **これらの要因がどのように関連しているかを説明する:** これらの要因は、主に人間活動による排出量増加によって引き起こされます。特に、火力発電や自動車の使用で発生する二酸化炭素などが挙げられます。

5. **結論を導き出す:** これらの情報を基に、地球温暖化の主な原因として、気温の上昇と大気中の二酸化炭素（CO2）の増加が挙げられることを示します。




</thinking><output>気候変動の一番大きな原因は、人間の活動により大量に排出される温室効果ガスの増加である。</output><end_of_turn><eos>


In [5]:
prompt = """<start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください。<end_of_turn>
<start_of_turn>user
人工知能の倫理的な問題点について、議論を深めるための質問をいくつか提案してください。<end_of_turn>
<start_of_turn>model
"""
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer)
outputs = model.generate(**input_ids,max_new_tokens=2048,streamer=streamer, top_k=2,top_p=0.95,do_sample=True, temperature=0.6,repetition_penalty=1.1)

<bos><start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください。<end_of_turn>
<start_of_turn>user
人工知能の倫理的な問題点について、議論を深めるための質問をいくつか提案してください。<end_of_turn>
<start_of_turn>model
<thinking>
## 人工知能の倫理的な問題点を議論するための質問：ステップバイステップで考える

1. **はじめに**
   - 人工知能が急速に進歩している中、その発展に伴う倫理的な課題は多岐にわたるものがあります。どのような倫理的問題点が特に注目されていますか？

2. **具体例を挙げる**
   - 例えば、AIが人間の仕事を奪い、失業率の上昇につながる可能性があることや、プライバシー侵害の可能性など、具体的な問題点を挙げることができます。

3. **それぞれの問題点に対する考え方を問う**
   - それぞれの問題点はどのように解決されるべきでしょうか？また、その解決策にはどのようなメリットとデメリットがありますか？

4. **結論を導き出す**
   - これらの問題点を総合的に分析し、人々の関心を集めるような論議を誘発する質問を作成します。



このステップに従って、「人工知能の倫理的な問題点を議論するための質問」というテーマについての質問を構成することができます。<end_of_turn><eos>


In [6]:
prompt = """<start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください。<end_of_turn>
<start_of_turn>user
日本の電柱は合計で何本ぐらいありそうですか
<end_of_turn>
<start_of_turn>model
"""
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer)
outputs = model.generate(**input_ids,max_new_tokens=2048,streamer=streamer, top_k=2,top_p=0.99,do_sample=True, temperature=0.6,repetition_penalty=1.1)

<bos><start_of_turn>system
あなたは親切なアシスタントです。ユーザーの質問に対して親切に回答してください。<end_of_turn>
<start_of_turn>user
日本の電柱は合計で何本ぐらいありそうですか
<end_of_turn>
<start_of_turn>model
<thinking>
## 日本の電柱の数を求める手順

1. **電柱の種類を特定する:** 電柱には、住宅用、公共用など様々な種類があります。ここでは、主に住宅用の電柱について考えます。

2. **住宅用電柱の数を推定する:** 住宅用電柱は、道路沿いに設置されているものと、敷地内に設置されているものが存在します。

3. **住宅用電柱の数を計算する:** 
    * 道路沿いの住宅用電柱数: 一般的に、1kmあたり約40本の電柱が設置されています。
    * 敷地に設置されている住宅用電柱数: 1軒あたりの電柱数は、5～8本程度です。ただし、地域や家の規模によって異なる可能性があります。
    * 総計として、道路沿いの電柱 + 敷地に設置されている電柱 = 約40本/km * 1軒当たりの電柱数 = 40～400本/軒（概算）

4. **総世帯数と平均的な家数から推定する:** 日本では、約7億6千万世帯が存在し、平均的な家数も約2人家族であると仮定します。

5. **推定結果をまとめる:**
    * 道路沿いの住宅用電柱数: 約40本/km * 1軒当たりの電柱数 = 40～400本/軒
    * 敷地に設置されている住宅用電柱数: 1軒あたりの電柱数 = 5～8本
    * 総計として、道路沿いの電柱 + 敷地に設置されている電柱 = 40～400本/軒 * 1軒当たりの電柱数 = 400～4000本/軒（概算）



この手順により、日本国内の住宅用電柱の数は、約400万本以上あることが推定されます。しかし、これはあくまで粗い推定であり、実際の電柱の本数がより多くなると予想されます。また、電気事業者や自治体による管理体制の違いなども考慮に入れる必要があります。<end_of_turn><eos>


In [8]:
prompt = """<start_of_turn>system
あなたはユーザーの要望に対応したReactとTailwindCSSを使ったプログラムを作成するTypeScriptエンジニアです。<end_of_turn>
<start_of_turn>user
電卓アプリを作成してApp.tsxに実装してください。
<end_of_turn>
<start_of_turn>model
"""
input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
streamer = TextStreamer(tokenizer)
outputs = model.generate(**input_ids,max_new_tokens=4096,streamer=streamer, top_k=2,top_p=0.99,do_sample=True, temperature=0.6,repetition_penalty=1.1)

<bos><start_of_turn>system
あなたはユーザーの要望に対応したReactとTailwindCSSを使ったプログラムを作成するTypeScriptエンジニアです。<end_of_turn>
<start_of_turn>user
電卓アプリを作成してApp.tsxに実装してください。
<end_of_turn>
<start_of_turn>model
<thinking>1. **プロジェクトのセットアップ:** Reactアプリケーションを構築するためには、Node.jsやYarnなどのパッケージマネージャが必要となります。また、TypeScript言語を使用したい場合は、TypeScript compilerも必要になります。

2. **コンポーネントの作成:** App.tsxファイルに、電卓機能を実装するためのコンポーネントを作成します。このコンポーネントは、ボタンやテキストボックスなどを含むUI要素を含みます。

3. **イベントハンドラの実装:** 各ボタンをクリックすると、そのボタンが押されたときに実行される処理を定義します。たとえば、加算ボタンを押すと、現在の値に加算する処理を実行し、減算ボタンを押すと、現在の値から引き算する処理を実行します。

4. **計算結果の表示:** 計算結果を表示するために、resultコンポーネントを定義します。これは、入力値と計算結果を保持し、それを画面上に表示します。

5. **テンプレートファイルの追加:** index.tsxファイルに、各コンポーネントをインポートし、それぞれのコンポーネントを呼び出します。

6. **コードのテスト:** テストフレームワーク（Jestなど）を使用して、コンポーネントの動作を確認します。例えば、各ボタンをクリックしても正しい計算が行われることを確認します。

7. **リファクタリング:** コードの可読性を向上させるために、コメントを追加したり、変数名を分かりやすく変更したりします。



</thinking><output>```typescript
import { useState } from 'react';

// Calculator component for app
const App = () => {
  const 